In [1]:
import numpy as np
import pandas as pd

In [2]:
perry_dir="/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
param_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"

ML_data_dir="/Users/hn/Documents/01_research_data/NASA/ML_data/"

In [43]:
set1_all_votes_Perry=pd.read_csv(perry_dir+"set1_all_votes.csv")
print (len(set1_all_votes_Perry.ID.unique()))

294


In [44]:
evaluation_set=pd.read_csv(param_dir+"evaluation_set.csv")
print (len(evaluation_set.ID.unique()), ",", evaluation_set.shape[0])

6340 , 6340


In [45]:
set1_all_votes_Perry=pd.merge(set1_all_votes_Perry, 
                              evaluation_set[["ID", "CropTyp", "ExctAcr"]], 
                              on=['ID'], how='left')

In [46]:
######## Toss Small Fields

print (len(set1_all_votes_Perry.ID.unique()))
set1_all_votes_Perry=set1_all_votes_Perry[set1_all_votes_Perry.ExctAcr>10]
print (len(set1_all_votes_Perry.ID.unique()))

294
219


# Hard Crops for Experts

In [47]:
print (len(sorted(set1_all_votes_Perry.CropTyp.unique())))
(sorted(set1_all_votes_Perry.CropTyp.unique()))

27


['alfalfa seed',
 'barley',
 'barley hay',
 'bean, dry',
 'bean, green',
 'bluegrass seed',
 'buckwheat',
 'canola',
 'carrot',
 'carrot seed',
 'corn seed',
 'corn, field',
 'corn, sweet',
 'grass hay',
 'grass seed',
 'market crops',
 'mint',
 'oat hay',
 'onion',
 'pea, dry',
 'pea, green',
 'potato',
 'triticale',
 'triticale hay',
 'wheat',
 'wheat fallow',
 'yellow mustard']

# Non-Expert, Survey 2

In [48]:
NE_S2_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/nonExpert_set2_fields/"
nonExpert_survey2_fields=pd.read_csv(NE_S2_dir+"nonExpert_survey2_fields.csv")

In [49]:
print (len(nonExpert_survey2_fields.ID.unique()))
print (nonExpert_survey2_fields.shape)
nonExpert_survey2_fields=nonExpert_survey2_fields[nonExpert_survey2_fields.ExctAcr>10]
print (nonExpert_survey2_fields.shape)

1429
(1429, 14)
(1022, 14)


In [51]:
print (len(sorted(nonExpert_survey2_fields.CropTyp.unique())))
((sorted(nonExpert_survey2_fields.CropTyp.unique())))

24


['barley',
 'barley hay',
 'bean, dry',
 'bean, green',
 'bluegrass seed',
 'buckwheat',
 'canola',
 'carrot',
 'corn seed',
 'corn, field',
 'corn, sweet',
 'grass hay',
 'grass seed',
 'market crops',
 'mint',
 'oat hay',
 'onion',
 'pea seed',
 'pea, dry',
 'pea, green',
 'potato',
 'triticale',
 'wheat',
 'yellow mustard']

In [55]:
nonExpert_survey2_fields[nonExpert_survey2_fields.ID.isin(list(set1_all_votes_Perry.ID))]

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county


# Expert Set 2 dammit

In [56]:
set2_all_responses=pd.read_csv(perry_dir+"set2_all_responses.csv")
set2_hand_picked_extended=pd.read_csv(perry_dir+"set2_hand_picked_extended.csv")

In [57]:
set2_all_responses.shape

(322, 10)

In [58]:
print (len(set2_hand_picked_extended.ID.unique()))
set2_hand_picked_extended.shape

322


(322, 14)

### Some of the handpicked fields were repetitions from Set 1. Drop them

In [59]:
set2_hand_picked_extended=set2_hand_picked_extended[~set2_hand_picked_extended.ID.isin(set1_all_votes_Perry.ID)]
set2_hand_picked_extended=set2_hand_picked_extended[set2_hand_picked_extended.ExctAcr>10]
set2_hand_picked_extended.shape

(139, 14)

In [60]:
len(set2_hand_picked_extended.ID.unique())+\
len(nonExpert_survey2_fields.ID.unique())+\
len(set1_all_votes_Perry[set1_all_votes_Perry.ExctAcr>=10].ID.unique())

1380

In [61]:
len(set1_all_votes_Perry[set1_all_votes_Perry.ExctAcr>=10].ID.unique())+\
len(set1_all_votes_Perry[set1_all_votes_Perry.ExctAcr<10].ID.unique())

219

In [62]:
experts_set1_and_2_NE_set2=pd.concat([set1_all_votes_Perry[["ID", "ExctAcr", "CropTyp"]],
                                      nonExpert_survey2_fields[["ID", "ExctAcr", "CropTyp"]],
                                      set2_hand_picked_extended[["ID", "ExctAcr", "CropTyp"]]
                                      ]
                                      )

print (experts_set1_and_2_NE_set2.shape)
print (len(experts_set1_and_2_NE_set2.ID.unique()))

(1380, 3)
1380


In [63]:
train_labels=pd.read_csv(ML_data_dir+"train_labels.csv")
train_labels=pd.merge(train_labels, 
                      evaluation_set[["ID", "CropTyp", "ExctAcr"]], 
                      on=['ID'], how='left')

train_labels=train_labels[train_labels.ExctAcr>=10]

In [68]:
train_labels[train_labels.ID.isin(list(experts_set1_and_2_NE_set2.ID))].shape

(1342, 4)

In [81]:
experts_set1_and_2_NE_set2[~experts_set1_and_2_NE_set2.ID.isin(list(train_labels.ID))]

,ID,ExctAcr,CropTyp
23,52269_WSDA_SF_2016,122.032301,bluegrass seed
24,51838_WSDA_SF_2016,185.294850,bluegrass seed
64,156767_WSDA_SF_2015,30.993346,triticale
72,109650_WSDA_SF_2018,63.541401,triticale
106,103177_WSDA_SF_2017,64.199500,canola
107,104671_WSDA_SF_2017,11.515580,canola
110,100439_WSDA_SF_2017,11.190091,canola
56,149714_WSDA_SF_2017,56.455126,"bean, dry"
124,3786_WSDA_SF_2016,39.791528,bluegrass seed
250,101090_WSDA_SF_2018,122.196713,"corn, field"
